In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytz


In [2]:
df = pd.read_csv('./Form-714-csv-files-June-2021/Part 3 Schedule 2 - Planning Area Hourly Demand.csv')
respondent_id = pd.read_csv('./Form-714-csv-files-June-2021/Respondent IDs.csv')
good_ids = respondent_id['respondent_id'].unique()[3:]
df = df[df['respondent_id'].isin(good_ids)]
hour_cols = [f'hour{i:02d}' for i in range(1, 25)]
df = df.loc[~(df[hour_cols] == 0).any(axis=1)]
df['timezone'] = df['timezone'].apply(lambda x: x.strip().upper())


In [3]:

timezones, counts = np.unique(df['timezone'].values.astype('str'), return_counts=True)
argsort = np.argsort(counts)[::-1]
print(counts[argsort])
print(timezones[argsort])
for ii in argsort:
    print(timezones[ii], counts[ii])

[130128 127461  73773  67919  49870  40665  38843  38732  11033  10197
   9469   6201   5476   4734   3312   3087   2559   2557   2191   2188
   2152   1658   1649    731    730    730    715    517    363    210
      3      2      1      1]
['EST' 'CST' 'PST' 'MST' 'CDT' 'CPT' 'PDT' 'EDT' 'DST' 'MDT' 'PPT' 'EPT'
 'HST' '2' 'CS' 'AKD' '1' '' 'MS' 'MPP' 'CD' 'EDS' 'AKS' 'MPT' 'EAS' 'CEN'
 'CDS' 'AST' 'E' 'ADT' 'CSR' '433' 'CTR' '206']
EST 130128
CST 127461
PST 73773
MST 67919
CDT 49870
CPT 40665
PDT 38843
EDT 38732
DST 11033
MDT 10197
PPT 9469
EPT 6201
HST 5476
2 4734
CS 3312
AKD 3087
1 2559
 2557
MS 2191
MPP 2188
CD 2152
EDS 1658
AKS 1649
MPT 731
EAS 730
CEN 730
CDS 715
AST 517
E 363
ADT 210
CSR 3
433 2
CTR 1
206 1


In [4]:
timezone_mapping = {
    "EST": "Etc/GMT+5", 
    "CST": "Etc/GMT+6", 
    "CS": "Etc/GMT+6", 
    "PST": "Etc/GMT+8", 
    "MST": "Etc/GMT+7", 
    "HST": "Etc/GMT+10",
    "AKST": "Etc/GMT+9",
    "EDT": "Etc/GMT+4",
    "CDT": "Etc/GMT+5",
    "CD": "Etc/GMT+5",
    "PDT": "Etc/GMT+7",
    "MDT": "Etc/GMT+6",
}
df['timezone'] = df['timezone'].apply(timezone_mapping.get)
df = df.dropna(subset=['timezone'])

In [5]:
timezones, counts = np.unique(df['timezone'].values.astype('str'), return_counts=True)
argsort = np.argsort(counts)[::-1]
print(counts[argsort])
print(timezones[argsort])
for ii in argsort:
    print(timezones[ii], counts[ii])

[182150 140970 106762  73773  38732   5476]
['Etc/GMT+5' 'Etc/GMT+6' 'Etc/GMT+7' 'Etc/GMT+8' 'Etc/GMT+4' 'Etc/GMT+10']
Etc/GMT+5 182150
Etc/GMT+6 140970
Etc/GMT+7 106762
Etc/GMT+8 73773
Etc/GMT+4 38732
Etc/GMT+10 5476


In [13]:
# Data Preparation
def prepare_data(df):
    # Melt hourly columns to long format
    id_vars = ['respondent_id', 'plan_date', 'timezone']
    hour_cols = [f'hour{ii:02}' for ii in range(1, 26)]
    df = df[hour_cols + id_vars]
    rename_dict = {a_col: int(a_col[-2:]) for a_col in hour_cols}
    df.rename(columns=rename_dict, inplace=True)
    df['plan_date'] = pd.to_datetime(df['plan_date'])
    df['plan_date'] = df.apply(
        lambda row: row['plan_date'].tz_localize(
            row['timezone']).tz_convert('UTC'),
        axis=1
    )
    df = pd.melt(df, 
                id_vars=id_vars,
                value_vars=np.arange(1, 25),
                var_name='hour',
                value_name='load')

    # Create local datetime
    df['utc_datetime'] = df['plan_date'] + pd.to_timedelta(df['hour'], unit='h')
    df = df.dropna(subset=['utc_datetime', 'load'])
    
    # Feature engineering
    df['utc_hour'] = df['utc_datetime'].dt.hour
    df['utc_dayofweek'] = df['utc_datetime'].dt.dayofweek
    df['utc_month'] = df['utc_datetime'].dt.month
    df['utc_year'] = df['utc_datetime'].dt.year
    
    # Cyclic encoding
    df['utc_hour_sin'] = np.sin(2 * np.pi * df['utc_hour'] / 24)
    df['utc_hour_cos'] = np.cos(2 * np.pi * df['utc_hour'] / 24)
    df['utc_dayofweek_sin'] = np.sin(2 * np.pi * df['utc_dayofweek'] / 7)
    df['utc_dayofweek_cos'] = np.cos(2 * np.pi * df['utc_dayofweek'] / 7)
    df['utc_month_sin'] = np.sin(2 * np.pi * df['utc_month'] / 12)
    df['utc_month_cos'] = np.cos(2 * np.pi * df['utc_month'] / 12)
    
    # Normalize year
    df['utc_year'] = (df['utc_year'] - df['utc_year'].mean()) / df['utc_year'].std()
    
    # Encode respondents
    df['respondent_id'] = df['respondent_id'].astype('category')
    df['respondent_idx'] = df['respondent_id'].cat.codes
    
    return df


In [18]:
df_prep = prepare_data(df)

/tmp/ipykernel_3252365/1144664126.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_dict, inplace=True)
/tmp/ipykernel_3252365/1144664126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plan_date'] = pd.to_datetime(df['plan_date'])
/tmp/ipykernel_3252365/1144664126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu